In [1]:
import serial as sr
import time as tm
import numpy as np
from core import connect
import threading as tr

In [2]:
busy = False 

In [3]:
# get open port info
portinfo = connect.getOpenPorts() 

# MUST set baudrate in pump "System Settings", and MUST match this rate:
baudrate=9600
# initiate Connection object with first open port
conn = connect.Connection(port = '/dev/tty.usbserial-A10MLXX5', baudrate = baudrate, x = 0, mode = 0,verbose = False)


In [4]:
def WDseq(v,r):
    print("withdrawing", v, "mL, at", r,"ml/min")

    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = -v              # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)       

    # Start pump
    conn.startPump()

In [5]:
def INseq(v,r):
    print("infusing", v, "mL, at", r,"ml/min")
    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = v              # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)       

    # Start pump
    conn.startPump()

In [6]:
def unClogSeq(v,r):
    print("begin unclogging")
    global busy
    global running
    
    pump_buffer_time = 2.5

    # big infuse
    INseq(v,r)
    tm.sleep(v / r * 60 + pump_buffer_time)

    # cycle 1
    WDseq(2.5,r)
    tm.sleep(2.5 / r * 60 + pump_buffer_time)
    INseq(2.5,r)
    tm.sleep(2.5 / r * 60 + pump_buffer_time)

    # cycle 2
    WDseq(2.5,r)
    tm.sleep(2.5 / r * 60 + pump_buffer_time)
    INseq(2.5,r)
    tm.sleep(2.5 / r * 60 + pump_buffer_time)

    # # cycle 3
    # WDseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
    # INseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
   
    busy    = False
    running = False


In [7]:
def isClog(pressure, clogLim, vinfed):
    global busy
    global t_clog
    global start_time


In [8]:
def runExp(v,r):
    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = -v               # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)       

    # Start pump
    conn.startPump()

In [9]:
arduino = sr.Serial(port='/dev/tty.usbmodem2101', baudrate=38400, timeout=1)
tm.sleep(2)  # Give some time for the connection to establish

In [10]:
# def read_arduino():
#     try:
#         arduino.flushInput()
#         data = arduino.readline()#.decode('utf-8').strip()
#         if data and data.isdigit():
#             return int(data)
#         else:
#             print("here")
#             return None
#     except:
#         # Catch any exception and ignore it
#         print("this here")
#         return None

def read_arduino():
    try:
        arduino.flushInput()  # Clear any junk in the input buffer
        data = arduino.readline().decode('utf-8').strip()
        return int(data)
    except:
        return None


In [11]:
# Function to read from Arduino in a separate thread
def read_arduino_thread():
    global latest_value
    while True:
        value = read_arduino()  # Your function to read from Arduino
        if value is not None:
            latest_value = value
        tm.sleep(0.25)  # Sleep to match Arduino sampling rate
        print(latest_value)


In [12]:
import threading
import time as tm
import numpy as np

global running
global t_clog
global start_time
global latest_value

running = False
busy = False

data_list = []
time_list = []
time_list_1 = [0]
pressureOfClog = 97 

t_clog = []
latest_value = None

exp_run_rate = 0.5 # ml/min
exp_vol      = 30 # ml
start_time   = tm.time()


# Start the Arduino reading thread
thread = threading.Thread(target=read_arduino_thread)
thread.daemon = True  # Daemon thread will automatically close when the main program exits
thread.start()

try:
    while True:
        value = latest_value
         # print("value is", value, " time is ", tm.time()-start_time)
        tm.sleep(0.1)
        
        if value is not None and value > 85:
            if not running and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)

            time_running = tm.time() - start_time
            P = value
            cL = pressureOfClog

            if P <= cL and not busy:
                t_clog = tm.time() - start_time_clog 
                vinfed = exp_run_rate / 60 * t_clog
                print("pressure fell to ", P, "which is below clog limit of" , cL)
                print("Time to clog is ", t_clog)
                busy = True
                unClogSeq(v=vinfed, r=50)
                time_list_1 = time_list_1.append(t_clog)

            round_time = np.round(tm.time() - start_time, 0)
        
except KeyboardInterrupt:
    print("Exiting...")
    conn.stopPump()
    


print("Data collection complete.")


102
102
102
102
running exp for volume 30  with flow rate of 0.5
102
102
2
2
2
2
2
2
99
98
98
99
100
99
100
98
100
99
99
100
99
100
100
100
101
100
101
100
99
100
99
100
100
100
100
100
100
100
100
99
99
101
100
100
101
100
100
100
99
100
100
99
101
100
100
99
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
99
99
100
100
99
100
100
100
100
100
100
100
100
100
101
100
100
100
100
100
100
100
100
100
100
100
100
101
101
100
100
101
100
100
100
101
101
100
100
100
100
100
100
100
101
99
100
101
101
99
100
100
100
100
101
100
100
100
100
100
101
99
100
100
76
81
pressure fell to  89 which is below clog limit of 97
Time to clog is  144.9310541152954
begin unclogging
infusing 1.2077587842941284 mL, at 50 ml/min
89
92
179
290
23
23
177
176
withdrawing 2.5 mL, at 50 ml/min
137
120
114
109
106
93
83
83
90
95
infusing 2.5 mL, at 50 ml/min
97
98
99
100
100
140
297
399
342
253
withdrawing 2.5 mL, at 50 ml/min
200
168
148
134
108
83
81
86
93
infusing 2.5 

AttributeError: 'NoneType' object has no attribute 'append'

97
99
98
98
99
99
99
100
99
99
99
100
99
100
100
101
100
99
101
101
100
101
101
101
101
100
101
101
102
102
102
102
101
102
102
102
101
101
102
101
102
102
102
102
102
102
102
102
102
101
102
101
101
101
102
101
102
101
101
102
102
101
101
102
102
102
102
102
102
101
102
102
102
102
102
102
101
102
101
102
102
102
102
102
102
102
102
102
102
102
101
102
102
102
102
103
102
102
102
101
102
103
103
102
102
103
102
103
102
102
102
101
102
102
102
103
103
102
102
103
103
102
103
102
103
102
103
102
103
103
103
103
102
103
102
103
102
103
102
102
102
102
102
102
102
103
103
102
103
102
103
103
103
103
103
103
102
103
102
102
103
103


In [ ]:
time_list_1

In [ ]:
global running
global t_clog
global start_time

running = False

data_list = []
time_list = []
pressureOfClog = 90 

t_clog = []

exp_run_rate = 5 # ml/min
exp_vol      = 30 # ml
start_time   = tm.time()

try:
    while True:
        value = read_arduino()
        print("value is", value, " time is ", tm.time()-start_time)
        tm.sleep(0.1)
        if value is not None and value > 85:
            if running is not True and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)

            time_running = tm.time()-start_time
            #time_list.append(time_running)
            #data_list.append(value)
            #pressmBar = (value - 100) * 2000/398
            

            # isClog(pressure = value, clogLim = pressureOfClog, vinfed= volume)
            P = value
            cL = pressureOfClog

            # print("pressure is ", P)
            # print("clog lim is ", cL)
            if P <= cL and not busy:
                t_clog = tm.time() - start_time_clog 
                vinfed = exp_run_rate / 60 * t_clog
                print("pressure fell to ", P, "which is below clog limit of" , cL)
                print("Time to clog is ", t_clog)
                busy = True
                unClogSeq(v=vinfed,r=50)

            # Print the latest value every few seconds (e.g., every 5 seconds)
            round_time = np.round(tm.time() - start_time,0)
            # if round_time % 3 == 0:  # 0.1s * 30 = 3 seconds
            #     print(f"Latest value (after {round_time} seconds): {value}")
            #     # print("press mbar is", pressmBar)
        
except KeyboardInterrupt:
    print("Exiting...")
    # Handle any cleanup if necessary here

print("Data collection complete.")
